In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import librosa

files_normaly = sorted(glob.glob(os.path.abspath('../input/train/*/normal/*.wav')))
files_anomaly = sorted(glob.glob(os.path.abspath('../input/train/*/anomaly/*.wav')))
files_test = sorted(glob.glob(os.path.abspath('../input/test/*.wav')))

normal = []
for file in files_normaly:
    y, sr = librosa.load(file, sr=None)
    normal.append(y)

normal = np.array(normal)

anomaly = []
for file in files_anomaly:
    y, sr = librosa.load(file, sr=None)
    anomaly.append(y)
anomaly = np.array(anomaly)

test = []
for file in files_test:
    y, sr = librosa.load(file, sr=None)
    test.append(y)
test = np.array(test)

In [2]:
melspec_normal = []
for n in normal:
    m = librosa.feature.melspectrogram(n, n_mels=256)
    m = librosa.power_to_db(m).astype(np.float32)
    melspec_normal.append(m)
melspec_normal = np.array(melspec_normal)

melspec_anomaly = []
for a in anomaly:
    m = librosa.feature.melspectrogram(a, n_mels=256)
    m = librosa.power_to_db(m).astype(np.float32)
    melspec_anomaly.append(m)
melspec_anomaly = np.array(melspec_anomaly)

melspec_test = []
for t in test:
    m = librosa.feature.melspectrogram(t, n_mels=256)
    m = librosa.power_to_db(m).astype(np.float32)
    melspec_test.append(m)
melspec_test = np.array(melspec_test)

In [3]:
train = np.concatenate([melspec_normal, melspec_anomaly])
train = train.reshape(train.shape[0],-1)
test = melspec_test.reshape(melspec_test.shape[0], -1)

target = np.concatenate([np.zeros(len(melspec_normal)), np.ones(len(melspec_anomaly))])

from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train, target, test_size=0.33, random_state=42)


In [4]:
from sklearn.svm import SVC

clf = SVC()

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_X)
train_X_scale = scaler.transform(train_X)
test_X_scale =scaler.transform(test_X)

In [6]:
clf.fit(train_X_scale, train_Y)

SVC()

In [7]:
clf.score(test_X_scale, test_Y)

0.9029304029304029

In [7]:
test_scale = scaler.transform(test)

In [8]:
pred = clf.predict(test_scale)

In [28]:
pred

array([0., 1., 1., ..., 0., 1., 1.])

In [29]:
sub = pd.read_csv('../input/sample_submission.csv', header=None)

In [30]:
sub[1] = pred.astype('int')

In [31]:
sub.shape

(3292, 2)

In [32]:
sub.to_csv('submission.csv', index=False, header=False)

In [33]:
sub

,0,1
0,machine01_000.wav,0
1,machine01_001.wav,1
2,machine01_002.wav,1
3,machine01_003.wav,0
4,machine01_004.wav,0
...,...,...
3287,machine16_097.wav,1
3288,machine16_098.wav,1
3289,machine16_099.wav,0
3290,machine16_100.wav,1
